<a href="https://colab.research.google.com/github/zzhenxi/Competitions/blob/main/%5BDACON%5D%EB%89%B4%EC%8A%A4%EA%B7%B8%EB%A3%B9%EB%B6%84%EB%A5%98%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [NOTE]
- dacon pretrained 모델 사용 금지 규칙이 있으므로 해당 코드는 제출에 사용되지 않았습니다.

# 데이터 불러오기

In [1]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/MyDrive/dacon

In [ ]:
# !unzip -qq "/content/drive/MyDrive/dacon/뉴스그룹분류경진대회.zip"

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/dacon/train.csv')

In [ ]:
train_df.head()

# 데이터 전처리
- 중복 / 결측치 확인
- \n 제거
- 특수문자 제거 (내용에 영향을 주는 경우도 있어 보류)
- 소문자 통합

In [ ]:
# 중복 / 결측치 확인 
train_df.duplicated().sum(), train_df.isnull().sum()

0

In [ ]:
# 특수문자 제거
import re

def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', inputString)
  return text_rmv

In [ ]:
train_df.text = train_df.text.apply(clean_text)

In [ ]:
# \n 제거
train_df.replace('\n', '', regex=True, inplace=True)

In [ ]:
# 소문자 통합
train_df.text = train_df.text.str.lower()

In [ ]:
# 전처리 된 data 저장
# train_df.to_csv('/content/drive/MyDrive/dacon/preprocessed_train.csv')

# 데이터 전처리2
* 전처리된 데이터가 구글 드라이브에 저장되어 있는 경우, 이 쉘을 실행

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/dacon/preprocessed_train.csv', lineterminator='\n')

In [6]:
train_df.head()

,Unnamed: 0,id,text,target
0,0,0,they were and even if washington might conside...,10
1,1,1,we run spacenews & views on our stareach bbs a...,14
2,2,2,not to worry the masons have been demonized a...,19
3,3,3,only brendan mckay or maybe arf would come to ...,17
4,4,4,help i am running some sample problems from or...,5


In [7]:
train_df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
train_df.head()

,id,text,target
0,0,they were and even if washington might conside...,10
1,1,we run spacenews & views on our stareach bbs a...,14
2,2,not to worry the masons have been demonized a...,19
3,3,only brendan mckay or maybe arf would come to ...,17
4,4,help i am running some sample problems from or...,5


[에러해결] tokenizer encoding을 하는 과정에서 nan값이 포함되었다고 하여, 결측치 제거

In [9]:
train_df.dropna(inplace=True)

# 모델링

### Set gpu enviroment

In [10]:
import torch

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [12]:
n_gpu

1

### parser data

In [13]:
# Get sentence data
sentences = train_df.text.to_list()
sentences[0]

'they were and even if washington might consider patty a bust id reworkthat trade in a minute  druce has been a complete and utter bust hereonly 5 goals'

In [14]:
# Get tag labels data
labels = train_df.target.to_list()
print(labels[0])

10


In [15]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
# 0:negative, 1: positive
tag2idx={'0': 0,
 '1': 1}

In [16]:
len(train_df.target.unique())

20

In [17]:
# 카테고리가 총 20개이므로 
tag2idx={}
for i in range(20):
  tag2idx[f'{i}'] = i

In [18]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

### sentencepiece model download
* 이미 다운로드 되었다면 실행할 필요 없음

In [ ]:
cd /content/drive/MyDrive/dacon

/content/drive/MyDrive/dacon


In [ ]:
pwd

'/content/drive/MyDrive/dacon'

In [ ]:
! wget https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model

--2022-04-08 05:10:23--  https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.15.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.15.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 798011 (779K) [binary/octet-stream]
Saving to: ‘xlnet-base-cased-spiece.model’

xlnet-base-cased-sp 100%[===================>] 779.31K  2.43MB/s    in 0.3s    

2022-04-08 05:10:24 (2.43 MB/s) - ‘xlnet-base-cased-spiece.model’ saved [798011/798011]



In [ ]:
ls # 모델이 잘 다운로드 되었는지 확인

preprocessed_train.csv  test.csv   xlnet-base-cased-spiece.model
sample_submission.csv   train.csv  뉴스그룹분류경진대회.zip


### Load tokenizer
* vocab_file에 sentencepiece 사용 

In [19]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.2 MB/s 


In [20]:
pip install transformers

     |████████████████████████████████| 4.0 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 77 kB 9.0 MB/s 
     |████████████████████████████████| 895 kB 67.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
cd /content/drive/MyDrive/dacon

/content/drive/MyDrive/dacon


In [22]:
vocabulary = 'xlnet-base-cased-spiece.model' # 상대경로임

In [23]:
max_len  = 64

In [24]:
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)

In [25]:
tokenizer = XLNetTokenizer(vocab_file=vocabulary, do_lower_case=False)

### set text input embedding


이 코드에 대한 내용은 xlnet githug repo
* https://github.com/zihangdai/xlnet/blob/master/data_utils.py
* https://github.com/zihangdai/xlnet/blob/master/classifier_utils.py  
 
에 정의되어 있다.   

생성되는 임배딩
- token id embedding
- mask embedding
- segment embedding

- Token type id 란? (segment id 라고도 부름)
BERT와 같은 모델에서는 입력으로 2개의 문장이 주어지는 경우가 있다. 입력으로 문장이 2개가 주어진 경우, 두 문장을 구분할 수 있도록 별도의 id를 부여한다. 2개의 문장이 입력으로 주어지면, 첫 번째 문장에 해당하는 token들의 인덱스에는 모두 0을, 두 번째 문장에 해당하는 token들의 인덱스에는 모두 1을 부여한다. 

In [26]:
max_len  = 64

full_input_ids = []
full_input_masks = []
full_segment_ids = []

In [27]:
SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

In [28]:
UNK_ID = tokenizer.encode("<unk>")[0]
CLS_ID = tokenizer.encode("<cls>")[0]
SEP_ID = tokenizer.encode("<sep>")[0]
MASK_ID = tokenizer.encode("<mask>")[0]
EOD_ID = tokenizer.encode("<eod>")[0]

In [29]:
for i, sentence in enumerate(sentences):
    # Tokenize sentence to token id list
    tokens_a = tokenizer.encode(sentence)
    
    # Trim the len of text -> token 갯수 조정
    if(len(tokens_a)>max_len-2):
        tokens_a = tokens_a[:max_len-2]
        # 2를 빼는 이유는 special token 때문인듯
        
    tokens = []
    segment_ids = [] # 필요한 이유?
    
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(SEG_ID_A) # 위치...?
        
    # Add <sep> token 
    tokens.append(SEP_ID) 
    segment_ids.append(SEG_ID_A)
    
    
    # Add <cls> token
    tokens.append(CLS_ID)
    segment_ids.append(SEG_ID_CLS) # 한 문장의 끝
    
    input_ids = tokens
    
    # The mask has 0 for real tokens and 1 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [0] * len(input_ids)

    # Zero-pad up to the sequence length at fornt 
    # max_len 만큼 '문장의 앞에' 제로패딩 
    if len(input_ids) < max_len:
        delta_len = max_len - len(input_ids)
        input_ids = [0] * delta_len + input_ids
        input_mask = [1] * delta_len + input_mask
        segment_ids = [SEG_ID_PAD] * delta_len + segment_ids

    assert len(input_ids) == max_len
    assert len(input_mask) == max_len
    assert len(segment_ids) == max_len
    
    full_input_ids.append(input_ids)
    full_input_masks.append(input_mask)
    full_segment_ids.append(segment_ids)
    
    if i==0: # 한개만 출력
        print("No.:%d"%(i))
        print("sentence: %s"%(sentence))
        print("input_ids:%s"%(input_ids))
        print("attention_masks:%s"%(input_mask))
        print("segment_ids:%s"%(segment_ids))
        print("\n")

No.:0
sentence: they were and even if washington might consider patty a bust id reworkthat trade in a minute  druce has been a complete and utter bust hereonly 5 goals
input_ids:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 63, 55, 21, 176, 108, 11048, 577, 327, 1524, 15300, 982, 24, 11417, 17, 1500, 17, 88, 3552, 4331, 750, 25, 24, 1901, 17, 7841, 11735, 51, 72, 24, 1009, 21, 17, 9035, 11417, 193, 6632, 306, 1667, 4, 3, 7739, 7739]
attention_masks:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
segment_ids:[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]




In [30]:
tokenizer.encode("<cls>"), tokenizer.encode("<unk>") # 왜 스페셜 토큰은 같은 임베딩 값일까? -> 나중에 업데이트 되어서 이지 않을까?

([7739, 11974, 23, 3151, 4, 3], [7739, 12287, 3151, 4, 3])

## set label embedding 

In [31]:
# Make label into id
# label값을 id로 바꿔주기 : 그런데 무슨 차이가 있는지는 모르겠다..
tags = [tag2idx[str(lab)] for lab in labels]
print(tags[0])

10


## train test split

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
'''
full_input_ids, full_input_masks, full_segment_ids는 모두 이중 리스트의 형태를 띈다.
[[토큰들], [토큰들], [토큰들]]
'''
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks, tr_segs, val_segs = train_test_split(full_input_ids, tags, full_input_masks, full_segment_ids, 
                                                            random_state=1004, test_size=0.3)

In [34]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(6412, 2748, 6412, 2748)

In [35]:
# 텐서로 변환
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [36]:
# dataloader에 넣어주기

In [37]:
# Set batch num
batch_num = 32

In [38]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [39]:
# Set token embedding, attention embedding, segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_segs, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
# if drop_last is true, the sampler will drop the last batch if its size would be less than batch_size
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num, drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks,val_segs, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

## train model

In [61]:
# Load model
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=len(tag2idx))

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [62]:
model.to(device)

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [63]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [65]:
# Cacluate train optimiazaion num
import math
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

## Set fine tuning method

In [66]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
# Since XLNet in 'pytorch_transformer' did not contian classifier layers
# FULL_FINETUNING = True need to set True
FULL_FINETUNING = True

In [67]:
from torch.optim import Adam

In [68]:
# 조금더 공부하기 

if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

## fine-tuning model

In [69]:
# model train 모드로 전환
model.train() 

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [48]:
import torch
import os
from tqdm import tqdm,trange

In [71]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs,b_labels = batch
        
        # forward pass
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        loss, logits = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
        

***** Running training *****
  Num examples = 6412
  Batch size = 32
  Num steps = 1005


Epoch:  20%|██        | 1/5 [01:25<05:43, 85.76s/it]

Train loss: 1.852290170788765


Epoch:  40%|████      | 2/5 [02:55<04:24, 88.14s/it]

Train loss: 1.1271612000465394


...

Train loss: 0.8256372323632241


Epoch:  80%|████████  | 4/5 [06:01<01:31, 91.58s/it]

Train loss: 0.5893865446001292


Epoch: 100%|██████████| 5/5 [07:35<00:00, 91.11s/it]

Train loss: 0.4058284097164869


## save model

In [72]:
xlnet_out_address = 'models/xlnet_out_model/tc02'

In [73]:
# Make dir if not exits
if not os.path.exists(xlnet_out_address):
        os.makedirs(xlnet_out_address)

In [74]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [75]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(xlnet_out_address, "pytorch_model.bin")
output_config_file = os.path.join(xlnet_out_address, "config.json")

In [76]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(xlnet_out_address)

('models/xlnet_out_model/tc02/spiece.model',)

load saved model

In [77]:
model = XLNetForSequenceClassification.from_pretrained(xlnet_out_address,num_labels=len(tag2idx))

In [78]:
model.to(device)

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

eval model

In [84]:
import numpy as np
from sklearn.metrics import classification_report


In [ ]:
# eval mode로 전환
model.eval()

In [80]:
# Set acc funtion
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [85]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_true = []
y_predict = []
print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs,b_labels = batch
    
    with torch.no_grad():
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]
    
    # Get textclassification predict result
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
#     print(tmp_eval_accuracy)
#     print(np.argmax(logits, axis=1))
#     print(label_ids)
    
    # Save predict and real label reuslt for analyze
    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
        
    for real_result in label_ids.tolist():
        y_true.append(real_result)

    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
   
    nb_eval_steps += 1
    
    
eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / len(val_inputs)
loss = tr_loss/nb_tr_steps 
result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'loss': loss}
report = classification_report(y_pred=np.array(y_predict),y_true=np.array(y_true))

# Save the report into file
output_eval_file = os.path.join(xlnet_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
        print("  %s = %s"%(key, str(result[key])))
        writer.write("%s = %s\n" % (key, str(result[key])))
        
    print(report)
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =2748
  Batch size = 32
***** Eval results *****
  eval_accuracy = 0.6859534206695779
  eval_loss = 1.1993423416170963
  loss = 0.4058284097164869
              precision    recall  f1-score   support

           0       0.39      0.47      0.43       116
           1       0.63      0.61      0.62       138
           2       0.62      0.69      0.65       151
           3       0.51      0.67      0.58       131
           4       0.75      0.64      0.69       140
           5       0.79      0.79      0.79       125
           6       0.83      0.78      0.80       158
           7       0.80      0.74      0.77       155
           8       0.73      0.66      0.70       125
           9       0.75      0.82      0.79       146
          10       0.80      0.80      0.80       143
          11       0.70      0.76      0.73       163
          12       0.70      0.58      0.63       133
          13       0.84      0.85      0.84       